In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Red_Hair"
cohort = "GSE207744"

# Input paths
in_trait_dir = "../../input/GEO/Red_Hair"
in_cohort_dir = "../../input/GEO/Red_Hair/GSE207744"

# Output paths
out_data_file = "../../output/preprocess/Red_Hair/GSE207744.csv"
out_gene_data_file = "../../output/preprocess/Red_Hair/gene_data/GSE207744.csv"
out_clinical_data_file = "../../output/preprocess/Red_Hair/clinical_data/GSE207744.csv"
json_path = "../../output/preprocess/Red_Hair/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptomic study on human skin samples: identification of actinic keratoses two risk classes."
!Series_summary	"Gene expression profile analysis allowed to identify 2 classes of AK."
!Series_overall_design	"A total of 72 tissue samples (24 NL, 23 L, 4 PL and 21 AK) were isolated from 24 patients. For each patient, samples were acquired on the lesion (L or AK), on the perilesional (PL) i.e. safety surgical margin area (often containing AK) and/or on the non-lesional (NL) parts of the elliptical surgical excision."
Sample Characteristics Dictionary:
{0: ['patient number: 001', 'patient number: 006', 'patient number: 016', 'patient number: 017', 'patient number: 018=026=045', 'patient number: 028', 'patient number: 029', 'patient number: 035=041', 'patient number: 048', 'patient number: 056', 'patient number: 057', 'patient number: 074', 'patient number: 075', 'patient number: 077', 'patient number: 082', 'patient number: 090', 'patient number: 0

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os

# 1. Determine if gene expression data is available
# Based on the background information, this is a transcriptomic study on human skin samples
# Therefore gene expression data should be available
is_gene_available = True

# 2.1 Data Availability for trait, age, and gender
# Looking at the sample characteristics dictionary:
# The dataset is about actinic keratosis skin lesions, not red hair.
# No information about red hair is available in this dataset.
# No age information is available in the sample characteristics
# No gender information is available in the sample characteristics

trait_row = None  # No red hair information in this dataset
age_row = None  # No age information available
gender_row = None  # No gender information available

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert trait information to binary values."""
    # Since there's no red hair data, this function is defined for completeness
    return None

def convert_age(value):
    """Convert age information to continuous values."""
    # No age information in this dataset, function defined for completeness
    return None

def convert_gender(value):
    """Convert gender information to binary values."""
    # No gender information in this dataset, function defined for completeness
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None  # Will be False

# Save the cohort information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (if trait_row is not None)
# Since trait_row is None, we'll skip this step
if trait_row is not None:
    # Load the actual clinical data that should be available from previous steps
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,  # This would be the actual data from previous steps
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the selected clinical features
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:", preview)
    
    # Save the selected clinical features to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)


A new JSON file was created at: ../../output/preprocess/Red_Hair/cohort_info.json


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print(gene_data.index[:20])


Index(['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107',
       '(+)E1A_r60_a135', '(+)E1A_r60_a20', '(+)E1A_r60_a22', '(+)E1A_r60_a97',
       '(+)E1A_r60_n11', '(+)E1A_r60_n9', '3xSLv1', 'A_19_P00315452',
       'A_19_P00315492', 'A_19_P00315493', 'A_19_P00315502', 'A_19_P00315506',
       'A_19_P00315518', 'A_19_P00315519', 'A_19_P00315529', 'A_19_P00315541'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Based on my biomedical knowledge, these identifiers don't appear to be standard human gene symbols
# The identifiers that start with "A_19_P" look like Agilent microarray probe IDs
# Others like "(+)E1A_r60_1" and "3xSLv1" are not standard gene symbols either
# These will need to be mapped to standard gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['GE_BrightCorner', 'DarkCorner', 'A_21_P0014386', 'A_33_P3396872', 'A_33_P3267760'], 'CONTROL_TYPE': ['pos', 'pos', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, 'NM_001105533', nan], 'GB_ACC': [nan, nan, nan, 'NM_001105533', nan], 'LOCUSLINK_ID': [nan, nan, nan, 79974.0, 54880.0], 'GENE_SYMBOL': [nan, nan, nan, 'CPED1', 'BCOR'], 'GENE_NAME': [nan, nan, nan, 'cadherin-like and PC-esterase domain containing 1', 'BCL6 corepressor'], 'UNIGENE_ID': [nan, nan, nan, 'Hs.189652', nan], 'ENSEMBL_ID': [nan, nan, nan, nan, 'ENST00000378463'], 'ACCESSION_STRING': [nan, nan, nan, 'ref|NM_001105533|gb|AK025639|gb|BC030538|tc|THC2601673', 'ens|ENST00000378463'], 'CHROMOSOMAL_LOCATION': [nan, nan, 'unmapped', 'chr7:120901888-120901947', 'chrX:39909128-39909069'], 'CYTOBAND': [nan, nan, nan, 'hs|7q31.31', 'hs|Xp11.4'], 'DESCRIPTION': [nan, nan, nan, 'Homo sapiens cadherin-like and PC-esterase domain containing 1 (CPED1), transcript variant 2, mRNA [NM_001105533

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns in gene_annotation contain gene identifiers and gene symbols
# Based on the preview, the column 'ID' appears to match the gene identifiers in gene_expression data
# The column 'GENE_SYMBOL' contains the gene symbols

# 2. Extract the gene mapping using the get_gene_mapping function from the library
gene_mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GENE_SYMBOL')

# Examine the mapping dataframe
print("Gene mapping dataframe preview:")
print(preview_df(gene_mapping_df))

# 3. Apply the gene mapping to convert probe-level measurements to gene-level expression
gene_data = apply_gene_mapping(gene_data, gene_mapping_df)

# Print the first 20 gene symbols after mapping to verify the process
print("First 20 gene symbols after mapping:")
print(gene_data.index[:20])

# Print the shape of the gene data after mapping
print(f"Gene data shape after mapping: {gene_data.shape}")


Gene mapping dataframe preview:
{'ID': ['A_33_P3396872', 'A_33_P3267760', 'A_32_P194264', 'A_23_P153745', 'A_21_P0014180'], 'Gene': ['CPED1', 'BCOR', 'CHAC2', 'IFI30', 'GPR146']}


First 20 gene symbols after mapping:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A1CF-2', 'A1CF-3', 'A2M', 'A2M-1',
       'A2M-AS1', 'A2ML1', 'A2MP1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AA06',
       'AAAS', 'AAAS-1', 'AACS', 'AACS-2', 'AACS-3', 'AACSP1'],
      dtype='object', name='Gene')
Gene data shape after mapping: (29222, 72)


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"First few normalized gene symbols: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Check if trait data is available by reading the JSON metadata
import json
with open(json_path, "r") as file:
    metadata = json.load(file)
    
is_trait_available = False
if cohort in metadata:
    is_trait_available = metadata[cohort].get("is_trait_available", False)

# Only proceed with clinical data processing if trait is available
if is_trait_available:
    # Load the clinical features from the saved file
    clinical_file_path = out_clinical_data_file
    if os.path.exists(clinical_file_path):
        clinical_features = pd.read_csv(clinical_file_path)
        print(f"Clinical features loaded from {clinical_file_path}")
        print(f"Clinical features shape: {clinical_features.shape}")
    
    # 2. Link the clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_features.T, normalized_gene_data)
    print(f"Linked data shape: {linked_data.shape}")
    print(f"First few columns: {list(linked_data.columns[:5])}")
    
    # 3. Handle missing values in the linked data
    trait_column = linked_data.columns[0]  # First column should be the trait
    print(f"Using trait column: {trait_column}")
    
    linked_data_processed = handle_missing_values(linked_data, trait_column)
    print(f"Shape after handling missing values: {linked_data_processed.shape}")
    
    # 4. Determine whether the trait and demographic features are severely biased
    is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_processed, trait_column)
    
    # 5. Conduct quality check and save the cohort information
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=True,
        is_biased=is_trait_biased, 
        df=unbiased_linked_data,
        note="Dataset contains gene expression data but was processed and found unsuitable for Red_Hair analysis."
    )
    
    # 6. Save the data if it's usable
    if is_usable:
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        # Save the data
        unbiased_linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print(f"Data quality check failed. The dataset is not suitable for association studies.")
else:
    print(f"No Red_Hair trait data available for cohort {cohort}. Cannot link clinical and genetic data.")
    # Create empty DataFrame with appropriate structure for validation
    empty_df = pd.DataFrame(columns=[trait])
    
    # Mark as unusable in final validation - using False for is_biased instead of None
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=False,
        is_biased=False,  # Using False instead of None to satisfy function requirements
        df=empty_df,
        note="No Red_Hair trait information available in this cohort."
    )

Normalized gene data shape: (20778, 72)
First few normalized gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1', 'A3GALT2', 'A4GALT', 'A4GNT']


Normalized gene data saved to ../../output/preprocess/Red_Hair/gene_data/GSE207744.csv
No Red_Hair trait data available for cohort GSE207744. Cannot link clinical and genetic data.
Abnormality detected in the cohort: GSE207744. Preprocessing failed.
